In [ ]:
# create a transcirptome using reference genome and annotated sequences
gffread -w csi.transcriptome.fa -g csi.chromsome.fa stringtie_merged.gtf

#create the index from the transcriptome that was created
salmon index -t csi.transcriptome.fa -i csi.transcriptome_index

#script for quantifying all of the samples downloaded from the citrus genome project
#!/bin/bash

cd /data/VALENCIA
# create a list so that a pattern can be used to run a script
ls *_1.fq.gz | cut -d'_'-f 1 > fastq.list

#iterate the quantification process for each sample
for file in fastq.list;
do
echo "Processing sample ${file}"
salmon quant -i /data/Salmon/csi.transcriptome_index -l A \
    -1 ${file}_1.fq.gz \
    -2 ${file}_2.fq.gz \ 
    -p 40 --validateMappings -o /data/Salmon/quants/${file}_quant
done

In [ ]:
# import the transcripts from Salmon for gene analysis w/ DESeq2
# created a file path
for (file in f){dir <- paste0("/RNA/quants/", f, "_quant)}

# read in the table with sample OD's which will be combined later with the transcript ocounts
samples <- read.csv("VALENCIA/phenotype.csv", header = TRUE)

# changed the Conditions columns to numeric values in order to comply w packages
samples["Condition"] <- c('1','1','1','2','2','2','3','3','3','4','4','4','5','5','5')

# create txt files to aling the sample ID and counts
 echo -e "TXNAME\tGENEID" > tx2gene.txt; awk '$3 == "transcript" {print $12"\t"$10}' stringtie_merged.gtf | sed 's/"//g;s/;//g' >> tx2gene.txt

# read in the txt file of the tscript and gene ID
tx2gene <- read.table("VALENCIA/tx2gene.txt")

# import quant/sf files by creating a vector with the file names

#files <- file.path(dir,"salmon", samples$sample, "quant.sf.gz")
files <- file.path(dir, "/quant.sf")

# use the df from earlier to create a list with all the file names
f <- samples[,1]
# rename the samples in "files" 
for(file in files){names(files) <- paste0(f)}

# import the quant.sf files
txi.salmon <- tximport(files, type = "salmon", tx2gene=tx2gene)


# create a DESeq data set
# create a df to align the samples with their condition
sampleTable <- data.frame(condition = factor(rep(c('1','2','3','4','5'), each = 3)))
rownames(sampleTable) <- colnames(txi.salmon$counts)
# create a DESeq data set with the sample counts ready for export
dds <-DESeqDataSetFromTximport(txi.salmon, sampleTable, ~condition)

# Run DESeq2 on the dataset 
dds1 <- DESeq(dds)

#build a table for each of the results you want to analyze
res_GHA_PFA <- results(dds1, contrast=c("condition","2","1"))
res_102_103 <- results(dds1, contrast=c("condition","2","3"))
res_102_108 <- results(dds1, contrast=c("condition","2","4"))
res_102_PV5 <- results(dds1, contrast=c("condition","2","5"))
res_103_108 <- results(dds1, contrast=c("condition","3","4"))
res_103_PV5 <- results(dds1, contrast=c("condition","3","5"))
res_108_PV5 <- results(dds1, contrast=c("condition","4","5"))


# Save the results to a csv file
# create a list of strings to save the df to
res <- c('res_GHA_PFA', 'res_102_103', 'res_102_108', 'res_102_PV5', 'res_103_108', 'res_103_PV5', 'res_108_PV5')

# create a list to iterate the dfs
res.list <- list(res_GHA_PFA, res_102_103, res_102_108, res_102_PV5, res_103_108, res_103_PV5, res_108_PV5)

# Order each of the lists by padj
lapply(res.list, function(x) res.list_sort <- x[order(x$padj),])

# write the dataframe to csv
for (i in 1:7){write.csv(res.list_order[i], paste("/RNA/VALENCIA/", res[i], ".csv", sep=" "), row.names = TRUE)}


In [ ]:
# create graphs for each of the dataframes
lapply(res.list, function(x) plotMA(x, ylim=c(-10,10)))

#save the graphs
for(i in 1:7){
    svg(paste0("/RNA/quants/", res[i], ".svg"), width = 7, height = 7)
    lapply(res.list, function(x) plotMA(x, ylim=c(-10,10)))
    
dev.off()
}

In [ ]:
# Compare significant genes
#subset all the padj values and place them into one list
DEG <- lapply(res.list, subset, padj <0.05)
#concatenate all of the significant genes
do.call(rbind, DEG)

In [ ]:
# Compare the significant genes
colnames(comp) <- c("comp1")
# create an list of all the gene names
res_names.list <- lapply(res.list, function(x) rownames(x))

#remove duplicated genes
DEG <- unique(DEG)

# create a vector of gene names
genes <- as.vector(rownames(DEG))

# compare the genes
sapply(genes, function(x) x %in% res.list[i])

# iterate this function and output to the list
comp <- data.frame()
for (i in 1:7){comp[i] <- cbind(lapply(genes, function(x) x %in% res_names.list[i])}

# change logical values to numeric
as.numeric(comp)

In [ ]:
# subset all the pval values 
PVAL <- lapply(res.list, subset, pvalue<0.05)

#concatenate the values
PVAL.list<- do.call(rbind, PVAL)

# output names as a vector
PVA_names <- lapply(PVAL, function(x) rownames(x))

# output gene names as a vector
genes1 <- as.vector(rownames(PVAL.list))

#remove duplicated genes
genes1 <- unique(genes1)

# create a deg column for the dataframe
comp1 <- data.frame(genes1)

#iterate an function to match the genes
# unlist PVA object to create a vector so is.element can be used
for (i in 2:8){comp1[i] <- is.element(genes1,unlist(PVA_names[i-1]))}

#convert data to numeric
cols <- sapply(comp1, is.logical)
comp1[,cols] <- lapply(comp1[,cols], as.numeric)

# find the total matched genes of the data. convert data to isnumeric
sum(comp1[i]) #sum of a column in the dataframe
Total<-colSums(comp1[2:8])


In [ ]:
install.packages('VennDiagram')
#create a venn diagram
# change the column names
colnames(comp1)<- c('geneID','GHA_PFA','102_103','102_108','102_PV5','103_108','103_PV5','108_PV5')

# create groups to put into venn diagram
col102 <- c(3:5)
col103 <- c(3,6,7)
colPV5 <- c(5,7,8)
# create multiple triple venn diagrams
venn(comp1[,102], snames = c('102-102','102-108','102-PV5'),counts = 1, zcolor = "style")
venn(comp1[,103], snames = c('102-102','102-108','102-PV5'),counts = 1, zcolor = "style")
venn(comp1[,PV5], snames = c('102-102','102-108','102-PV5'),counts = 1, zcolor = "style")

In [ ]:
lapply(PVAL, function(x) PVAL_order <- x[order(x$pvalue),])

In [ ]:
# Separate upregulated DEG and downregulated DEG
# check to see if the logfold2change provided by pipeline is which ratio
# retrieve normalized values of each sample
dds <- estimateSizeFactors(dds)
counts(dds, normalized=TRUE)

#find sum each condition for  a gene
Cs1g01170 <- head(counts(dds, normalized=TRUE), n=1)

sum(Cs1g01170[1:3])
sum(Cs1g01170[4:6])
sum(Cs1g01170[7:9])
sum(Cs1g01170[10:12])
sum(Cs1g01170[13:15])

# compared the values with the ones received from the pipeline, which matched
# subset the upregulated 102 genes at p < 0.05 FC >= 1
upreg102 <- lapply(PVAL[2:4], subset, log2FoldChange >=1)
#subset the downregulated 102 genes at p <0.05 FC <= -1
dowreg102 <- lapply(PVAL[2:4], subset, log2FoldChange <=-1)

#subset upregulated 103 genes 
upreg103 <- lapply(PVAL[2], subset, log2FoldChange <=-1)
upreg103[2:3] <- lapply(PVAL[5:6], subset, log2FoldChange >=1)
#subset downregulated 103 genes
downreg103 <- lapply(PVAL[2], subset, log2FoldChange >=1)
downreg103[2:3] <- lapply(PVAL[5:6], subset, log2FoldChange <=-1)

#subset upregulated 108 genes
upreg108 <- lapply(PVAL[3], subset, log2FoldChange <= -1)
upreg108[2] <- lapply(PVAL[5], subset, log2FoldChange <= -1)
upreg108[3] <- lapply(PVAL[7], subset, log2FoldChange >= 1)
#subset downregulated 108 genes
downreg108 <- lapply(PVAL[3], subset, log2FoldChange >= 1)
downreg108[2] <- lapply(PVAL[5], subset, log2FoldChange >= 1)
downreg108[3] <- lapply(PVAL[7], subset, log2FoldChange <= -1)

In [ ]:
# create a csv  of all the regulated expressed genes
# 1) sort the original file of genes alphanumerically by gene ID
# read res.list as a data frame
as.data.frame(res.list[i])
# read the rownames of the data frame
head(rownames(as.data.frame(res.list[i])))
# order the geneID 
install.packages('gtools')
library(gtools)
mixedorder(rownames(as.data.frame(res.list[i])))

# 2) isolate the values in reg.list 
head(subset(as.data.frame(res.list[i]), select = c('log2FoldChange','pvalue')))
# apply this to each list in res.list
l<-(lapply(res.list, function(x) (subset(as.data.frame(x),select = c('log2FoldChange','pvalue')))))


# 3) output to an empty dataframe concatenated. geneID|log2FoldChange[i]|pvalue[i]|...
reggenes <- cbind(as.data.frame(l[1:7]))
# change the column names so that they're easier to manage
colnames(reggenes) <- c('log2FC1','pval1','log2FC2','pval2','log2FC3','pval3','log2FC4','pval4','log2FC5','pval5','log2FC6','pval6','log2FC7','pval7')


#4) subset the up and down regulated items
# subset them to have pvalues less than 0.05
head(subset(reggenes, (pval1 < 0.05)))
head(subset(reggenes, (pval2 < 0.05)))

#5) subset them to which log2FC values are <= -1 for sample 2 and >= 1 for sample 5 and 6
# use logical conjunction
subset(reggenes, (log2FC2 <=-1 & pval2 <0.05) & (log2FC5 >=1 & pval5 < 0.05) & (log2FC6 >=1 & pval6 <0.05), select= c('log2FC2','pval2','log2FC5','pval5','log2FC6','pval6'))
upregulated <- subset(reggenes, (log2FC2 <=-1 & pval2 <0.05) & (log2FC5 >=1 & pval5 < 0.05) & (log2FC6 >=1 & pval6 <0.05), select= c('log2FC2','pval2','log2FC5','pval5','log2FC6','pval6'))
#change column names
colnames(upregulated) <- c('102log2FC','102pval','108log2FC','108pval','PV5log2FC','PV5pval')
#subset downregulated
subset(reggenes, (log2FC2 >=1 & pval2 <0.05) & (log2FC5 <=-1 & pval5 < 0.05) & (log2FC6 <=-1 & pval6 <0.05), select= c('log2FC2','pval2','log2FC5','pval5','log2FC6','pval6'))
downregulated <- subset(reggenes, (log2FC2 >=-1 & pval2) & (log2FC5 <=-1 & pval5) & (log2FC6 <=-1 & pval6), select= c('log2FC2','pval2','log2FC5','pval5','log2FC6','pval6'))
colnames(downregulated) <- c('102log2FC','102pval','108log2FC','108pval','PV5log2FC','PV5pval')

#5a) output to csv
write.csv(upregulated, file = "/RNA/quants/103upreg.csv")
write.csv(downregulated, file = "/RNA/quants/103downreg.csv")
write.csv(reggenes, file= "/RNA/quants/ordergenes")

In [ ]:
#6) subset the upregulated and downregulated genes of the other comparisons
upreggha_pfa<-subset(reggenes, (log2FC1 >= 1 & pval1 < 0.05), select = c('log2FC1', 'pval1'))
downreggha_pfa<-subset(reggenes, (log2FC1 <=-1 & pval1 < 0.05), select = c('log2FC1', 'pval1'))

upreg102_103<-subset(reggenes, (log2FC2 >= 1 & pval2 < 0.05), select = c('log2FC2', 'pval2'))
downreg102_103<-subset(reggenes, (log2FC2 <=-1 & pval2 < 0.05), select = c('log2FC2', 'pval2'))

upreg102_108<-subset(reggenes, (log2FC3 >= 1 & pval3 < 0.05), select = c('log2FC3', 'pval3'))
downreg102_108<-subset(reggenes, (log2FC3 <=-1 & pval3 < 0.05), select = c('log2FC3', 'pval3'))

upreg102_PV5<-subset(reggenes, (log2FC4 >= 1 & pval4 < 0.05), select = c('log2FC4', 'pval4'))
downreg102_PV5<-subset(reggenes, (log2FC4 <=-1 & pval4 < 0.05), select = c('log2FC4', 'pval4'))

upreg103_108<-subset(reggenes, (log2FC5 >= 1 & pval5 < 0.05), select = c('log2FC5', 'pval5'))
downreg103_108<-subset(reggenes, (log2FC5 <=-1 & pval5 < 0.05), select = c('log2FC5', 'pval5'))

upreg103_PV5<-subset(reggenes, (log2FC6 >= 1 & pval6 < 0.05), select = c('log2FC6', 'pval6'))
downreg103_PV5<-subset(reggenes, (log2FC6 <=-1 & pval6 < 0.05), select = c('log2FC6', 'pval6'))

upreg108_PV5<-subset(reggenes, (log2FC7 >= 1 & pval7 < 0.05), select = c('log2FC7', 'pval7'))
downreg108_PV5<-subset(reggenes, (log2FC7 <=-1 & pval7 < 0.05), select = c('log2FC7', 'pval7'))

In [ ]:
#output the data to csv files
write.csv(upreggha_pfa, file = "/RNA/quants/upregGHA_PFA.csv")
write.csv(downreggha_pfa, file = "/RNA/quants/downregGHA_PFA.csv")

write.csv(upreg102_103, file = "/RNA/quants/upreg102_103.csv")
write.csv(downreg102_103, file = "/RNA/quants/downreg102_103.csv")

write.csv(upreg102_108, file = "/RNA/quants/upreg102_108.csv")
write.csv(downreg102_108, file = "/RNA/quants/downreg102_108.csv")

write.csv(upreg102_PV5, file = "/RNA/quants/upreg102_PV5.csv")
write.csv(downreg102_PV5, file = "/RNA/quants/downreg102_PV5.csv")

write.csv(upreg103_108, file = "/RNA/quants/upreg103_108.csv")
write.csv(downreg103_108, file = "/RNA/quants/downreg103_108.csv")

write.csv(upreg103_PV5, file = "/RNA/quants/upreg103_PV5.csv")
write.csv(downreg103_PV5, file = "/RNA/quants/downreg103_PV5.csv")

write.csv(upreg108_PV5, file = "/RNA/quants/upreg108_PV5.csv")
write.csv(downreg108_PV5, file = "/RNA/quants/downreg108_PV5.csv")